In [ ]:
!pip install ultralytics opencv-python-headless

In [57]:
import cv2
import numpy as np
from ultralytics import YOLO
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor


model = YOLO('/yolov8l-face.pt')

In [ ]:
def apply_blur(image, coords):
    for (x1, y1, x2, y2) in coords:
        face = image[y1:y2, x1:x2]
        face = cv2.GaussianBlur(face, (99, 99), 30)
        image[y1:y2, x1:x2] = face
    return image

In [ ]:
def process_image(image_path):
    image = cv2.imread(image_path)
    small_image = cv2.resize(image, (image.shape[1] // 2, image.shape[0] // 2))
    return image, small_image

In [ ]:
def detect_faces(small_image):
    results = model(small_image)
    coords = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            coords.append((x1 * 2, y1 * 2, x2 * 2, y2 * 2))

    return coords


In [ ]:

def blur_faces(image_path, output_path):
    image, small_image = process_image(image_path)
    coords = detect_faces(small_image)
    blurred_image = apply_blur(image, coords)
    cv2.imwrite(output_path, blurred_image)

In [ ]:
blur_faces('/faces.jpg', '/output.jpg')


0: 960x832 9 faces, 7467.9ms
Speed: 11.3ms preprocess, 7467.9ms inference, 1.6ms postprocess per image at shape (1, 3, 960, 832)
